In [ ]:
from openai import AsyncAzureOpenAI, AsyncOpenAI
from agents import set_default_openai_client, Runner, set_tracing_export_api_key, Agent, OpenAIChatCompletionsModel, trace
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
openai_client = AsyncAzureOpenAI(
api_key=os.getenv("AZURE_OPENAI_API_KEY"),
api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)
set_default_openai_client(openai_client)

model = OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)

In [ ]:
banking_assistant = Agent(
   name="Banking Assistant",
   instructions="You are a helpful banking assistant. Be concise and professional.",
   model=model
)
set_tracing_export_api_key(os.getenv("OPENAI_API_KEY"))

with trace("prueba ejemplo"):
    respuesta = await Runner.run(banking_assistant, input="Hola")
    print(respuesta.final_output)

Puedes revisar la traza de la ejecucion en: https://platform.openai.com/logs?api=traces

¿Como funciona con otros modelos?

In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"



deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key="deepseek_api_key")
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key="google_api_key")
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key="groq_api_key")

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)



## Workflow de Encadenado de Prompts (Prompt Chaining)

El **encadenado de prompts** es una técnica donde la salida de un prompt se utiliza como entrada para el siguiente, creando un flujo de procesamiento secuencial.

### Arquitectura Básica

```
Usuario → Prompt 1 → LLM → Resultado 1
                              ↓
                         Prompt 2 → LLM → Resultado 2
                                            ↓
                                       Prompt 3 → LLM → Resultado Final
```

### Componentes Clave

1. **Prompts Especializados**: Cada paso tiene un propósito específico (análisis, extracción, transformación, síntesis)
2. **Paso de Contexto**: La salida de cada etapa alimenta la siguiente
3. **Agentes**: Cada prompt puede ser manejado por un agente especializado

### Ventajas

- ✅ Mayor precisión en tareas complejas
- ✅ Mejor control sobre el flujo de procesamiento
- ✅ Facilita debugging al aislar cada etapa
- ✅ Permite especialización de prompts

### Ejemplo Aplicado

```
Consulta Usuario → Agente Clasificador → Agente Especializado → Agente Validador → Respuesta Final
```

In [ ]:
instrucciones_analizador = """Eres un analista de consultas bancarias. 
    Analiza la consulta del usuario y proporciona un contexto detallado sobre:
    - El tema principal de la consulta
    - Los puntos clave que deben abordarse
    - El tipo de información que el usuario espera
    
    Sé específico y estructurado."""

instrucciones_procesador= """Eres un asistente bancario especializado.
    Basándote en el análisis previo, proporciona una respuesta completa y detallada.
    Incluye ejemplos, procedimientos y toda la información relevante.
    Sé profesional y exhaustivo."""

instrucciones_validador="""Eres un validador de respuestas bancarias.
    Revisa que la respuesta sea profesional, clara y completa.
    Si está bien, devuelve la respuesta tal cual.
    Si necesita mejoras, optimízala manteniendo el contenido técnico."""

model = OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)

# Agente 1: Analizador de Consultas
analyzer_agent = Agent(name="Query Analyzer",instructions=instrucciones_analizador,model=model)

# Agente 2: Procesador Principal
processor_agent = Agent(name="Main Processor",instructions=instrucciones_procesador,model=model)

# Agente 3: Validador Final
validator_agent = Agent(name="Response Validator",instructions=instrucciones_validador,model=model)

# Función de Encadenado (sin enrutamiento)
async def prompt_chaining_workflow(user_query: str):
    print(f"📥 Consulta del usuario: {user_query}\n")
    
    # PASO 1: Análisis
    print("🔍 Paso 1: Analizando consulta...")
    analysis_result = await Runner.run(analyzer_agent, input=user_query)
    analysis = analysis_result.final_output
    print(f"✅ Análisis completado\n")
    
    # PASO 2: Procesamiento Principal
    print("🎯 Paso 2: Generando respuesta detallada...")
    processing_prompt = f"""Consulta del usuario: {user_query}
    
    Análisis previo: {analysis}

    Proporciona una respuesta completa basándote en este análisis."""
    
    processor_result = await Runner.run(processor_agent, input=processing_prompt)
    processor_response = processor_result.final_output
    print(f"✅ Respuesta generada\n")
    
    # PASO 3: Validación
    print("✔️ Paso 3: Validando respuesta final...")
    validation_prompt = f"""Consulta original: {user_query}
    
    Respuesta generada: {processor_response}

    Valida y optimiza esta respuesta si es necesario."""
    
    final_result = await Runner.run(validator_agent, input=validation_prompt)
    print(f"✅ Validación completada\n")
    
    # Resultado final
    print("=" * 60)
    print("📤 RESPUESTA FINAL:")
    print("=" * 60)
    print(final_result.final_output)
    
    return final_result.final_output

# Ejemplo de uso
await prompt_chaining_workflow("¿Cómo puedo consultar mi saldo?")


## Workflow de Enrutamiento con LLM (Routing Pattern)

El **enrutamiento con LLM** es una arquitectura donde un agente clasificador analiza la consulta del usuario y la dirige al agente especializado más apropiado.

### Arquitectura Básica

```
Usuario → Agente Clasificador → Decisión de Ruta
                                       ↓
                    ┌──────────────────┼──────────────────┐
                    ↓                  ↓                  ↓
            Agente Especialista 1  Agente Especialista 2  Agente Especialista N
                    ↓                  ↓                  ↓
                    └──────────────────┴──────────────────┘
                                       ↓
                               Respuesta Final
```

### Componentes Clave

1. **Agente Clasificador**: Analiza la intención del usuario y categoriza la consulta
2. **Lógica de Enrutamiento**: Dirige al agente especializado según la clasificación
3. **Agentes Especializados**: Expertos en dominios específicos (saldos, transferencias, préstamos, etc.)
4. **Respuesta Unificada**: El sistema devuelve una respuesta coherente al usuario

### Ventajas

- ✅ Especialización por dominio
- ✅ Mejor calidad de respuestas
- ✅ Escalabilidad (agregar nuevos especialistas fácilmente)
- ✅ Optimización de recursos (cada agente usa el modelo/prompt adecuado)

### Diferencia con Prompt Chaining

| Prompt Chaining | Routing Pattern |
|-----------------|-----------------|
| Procesamiento secuencial | Procesamiento paralelo/selectivo |
| Todos los pasos se ejecutan | Solo se ejecuta el agente seleccionado |
| Flujo lineal | Flujo condicional |

### Ejemplo Aplicado

```
"¿Cómo consulto mi saldo?" → Clasificador → "saldo" → Balance Specialist → Respuesta
"¿Cómo hago una transferencia?" → Clasificador → "transferencia" → Transfer Specialist → Respuesta
```

In [ ]:
# Agente Clasificador (Router)
router_agent = Agent(
    name="Query Router",
    instructions="""Eres un router inteligente de consultas bancarias.
    Analiza la consulta del usuario y clasifícala en UNA de estas categorías:
    - balance: consultas sobre saldo, estado de cuenta, movimientos
    - transfer: consultas sobre transferencias, envío de dinero
    - loan: consultas sobre préstamos, créditos
    - card: consultas sobre tarjetas de crédito o débito
    - general: otras consultas bancarias
    
    Responde ÚNICAMENTE con la categoría exacta (balance, transfer, loan, card o general).
    No agregues explicaciones ni texto adicional.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

# Agentes Especializados
balance_specialist = Agent(
    name="Balance Specialist",
    instructions="""Eres un experto en consultas de saldo y estado de cuenta.
    Proporciona información clara sobre:
    - Cómo consultar saldos (app móvil, web, cajeros)
    - Interpretación de movimientos
    - Diferencia entre saldo disponible y contable
    - Resolución de discrepancias
    Sé profesional y específico.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

transfer_specialist = Agent(
    name="Transfer Specialist",
    instructions="""Eres un experto en transferencias bancarias.
    Proporciona información detallada sobre:
    - Tipos de transferencias (mismo banco, otros bancos, internacionales)
    - Procedimientos paso a paso
    - Límites y comisiones
    - Tiempos de procesamiento
    - Requisitos de seguridad
    Sé claro y exhaustivo.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

loan_specialist = Agent(
    name="Loan Specialist",
    instructions="""Eres un experto en préstamos y créditos bancarios.
    Proporciona información sobre:
    - Tipos de préstamos disponibles
    - Requisitos y documentación
    - Tasas de interés y plazos
    - Proceso de solicitud
    - Consulta de préstamos activos
    Sé profesional y detallado.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

card_specialist = Agent(
    name="Card Specialist",
    instructions="""Eres un experto en tarjetas de crédito y débito.
    Proporciona información sobre:
    - Tipos de tarjetas disponibles
    - Activación y bloqueo
    - Consulta de movimientos y estados de cuenta
    - Beneficios y recompensas
    - Reportar pérdida o robo
    Sé claro y completo.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

general_specialist = Agent(
    name="General Banking Assistant",
    instructions="""Eres un asistente bancario general.
    Proporciona información sobre:
    - Servicios bancarios generales
    - Horarios y sucursales
    - Apertura de cuentas
    - Atención al cliente
    - Otras consultas no especializadas
    Sé amable y profesional.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

# Mapeo de categorías a agentes
ROUTING_MAP = {
    "balance": balance_specialist,
    "transfer": transfer_specialist,
    "loan": loan_specialist,
    "card": card_specialist,
    "general": general_specialist
}

# Workflow de Enrutamiento
async def routing_workflow(user_query: str):
    print(f"📥 Consulta del usuario: {user_query}\n")
    
    # PASO 1: Clasificación (Routing)
    print("🔀 Paso 1: Enrutando consulta...")
    routing_result = await Runner.run(router_agent, input=user_query)
    category = routing_result.final_output.strip().lower()
    print(f"✅ Categoría identificada: {category}\n")
    
    # PASO 2: Selección del agente especializado
    specialist_agent = ROUTING_MAP.get(category, general_specialist)
    print(f"🎯 Paso 2: Redirigiendo a {specialist_agent.name}...\n")
    
    # PASO 3: Procesamiento por el especialista
    specialist_result = await Runner.run(specialist_agent, input=user_query)
    
    # Resultado final
    print("=" * 60)
    print("📤 RESPUESTA FINAL:")
    print("=" * 60)
    print(specialist_result.final_output)
    
    return {
        "category": category,
        "specialist": specialist_agent.name,
        "response": specialist_result.final_output
    }

# Ejemplos de uso
print("🔹 EJEMPLO 1: Consulta de saldo")
print("-" * 60)
await routing_workflow("¿Cómo puedo consultar mi saldo?")

# print("\n\n🔹 EJEMPLO 2: Consulta de transferencia")
# print("-" * 60)
# await routing_workflow("¿Cuánto tiempo tarda una transferencia internacional?")

# print("\n\n🔹 EJEMPLO 3: Consulta de tarjeta")
# print("-" * 60)
# await routing_workflow("¿Cómo bloqueo mi tarjeta de crédito?")

In [ ]:
from pydantic import BaseModel, Field

# Definir el modelo de salida estructurado
class IntentionDetection(BaseModel):
    """Modelo para la detección y clasificación de intenciones del usuario"""
    intention_detected: bool = Field(
        description="Indica si se detectó una intención bancaria válida"
    )
    category: str = Field(
        description="Categoría de la consulta: balance, transfer, loan, card o general"
    )
    confidence: str = Field(
        description="Nivel de confianza en la clasificación: high, medium, low"
    )
    reason: str = Field(
        description="Breve explicación de por qué se clasificó en esta categoría"
    )

# Agente Router con Output Parsing
structured_router_agent = Agent(
    name="Structured Query Router",
    instructions="""Eres un router inteligente de consultas bancarias.
    Analiza la consulta del usuario y determina:
    
    1. Si la consulta está relacionada con servicios bancarios (intention_detected: true/false)
    2. Clasifícala en UNA de estas categorías:
       - balance: consultas sobre saldo, estado de cuenta, movimientos
       - transfer: consultas sobre transferencias, envío de dinero
       - loan: consultas sobre préstamos, créditos
       - card: consultas sobre tarjetas de crédito o débito
       - general: otras consultas bancarias válidas
    
    3. Indica tu nivel de confianza (high/medium/low)
    4. Explica brevemente tu decisión
    
    Si la consulta NO es bancaria (ej: "¿cómo está el clima?"), marca intention_detected como false.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client),
    output_type=IntentionDetection  # ✨ Aquí activamos el output parsing
)

# Workflow mejorado con Output Parsing
async def enhanced_routing_workflow(user_query: str):
    print(f"📥 Consulta del usuario: {user_query}\n")
    
    # PASO 1: Clasificación estructurada
    print("🔀 Paso 1: Analizando y clasificando consulta...")
    routing_result = await Runner.run(structured_router_agent, input=user_query)
    
    # ✨ La salida ahora es un objeto Pydantic tipado
    intention: IntentionDetection = routing_result.final_output
    
    print(f"✅ Análisis completado:")
    print(f"   • Intención detectada: {'✓' if intention.intention_detected else '✗'}")
    print(f"   • Categoría: {intention.category}")
    print(f"   • Confianza: {intention.confidence}")
    print(f"   • Razón: {intention.reason}\n")
    
    # PASO 2: Validación y enrutamiento
    if not intention.intention_detected:
        print("⚠️ Consulta fuera del dominio bancario. No se enruta a especialista.\n")
        return {
            "intention_detected": False,
            "category": intention.category,
            "response": "Lo siento, solo puedo ayudarte con consultas bancarias. ¿Tienes alguna pregunta sobre tu cuenta, transferencias, préstamos o tarjetas?"
        }
    
    # PASO 3: Selección del especialista
    specialist_agent = ROUTING_MAP.get(intention.category, general_specialist)
    print(f"🎯 Paso 2: Redirigiendo a {specialist_agent.name}...\n")
    
    # PASO 4: Procesamiento por el especialista
    specialist_result = await Runner.run(specialist_agent, input=user_query)
    
    # Resultado final
    print("=" * 60)
    print("📤 RESPUESTA FINAL:")
    print("=" * 60)
    print(specialist_result.final_output)
    
    return {
        "intention_detected": intention.intention_detected,
        "category": intention.category,
        "confidence": intention.confidence,
        "reason": intention.reason,
        "specialist": specialist_agent.name,
        "response": specialist_result.final_output
    }

# 🔹 EJEMPLOS DE USO

# print("🔹 EJEMPLO 1: Consulta bancaria válida")
# print("-" * 60)
# result1 = await enhanced_routing_workflow("¿Cómo puedo consultar mi saldo?")
# with trace("prueba intencion no posible"):
#     print("\n\n🔹 EJEMPLO 2: Consulta fuera del dominio bancario")
#     print("-" * 60)
#     result2 = await enhanced_routing_workflow("¿Cómo está el clima hoy?")

# print("\n\n🔹 EJEMPLO 3: Consulta ambigua")
# print("-" * 60)
# result3 = await enhanced_routing_workflow("Necesito dinero urgente")

## Workflow de Paralelización (Parallel Pattern)

La **paralelización** es una arquitectura donde múltiples agentes trabajan simultáneamente en diferentes aspectos de una tarea, y luego sus resultados se agregan en una respuesta unificada.

### Arquitectura Básica

```
                            Usuario
                              ↓
                       Coordinador
                              ↓
        ┌─────────────────────┼─────────────────────┐
        ↓                     ↓                     ↓
   Agente 1              Agente 2              Agente 3
  (Asunto)            (Cuerpo)              (Firma)
        ↓                     ↓                     ↓
        └─────────────────────┴─────────────────────┘
                              ↓
                         Agregador
                              ↓
                      Email Completo
```

### Componentes Clave

1. **Coordinador**: Divide la tarea en subtareas independientes
2. **Agentes Paralelos**: Trabajan simultáneamente sin dependencias entre sí
3. **Agregador**: Combina los resultados en una salida coherente

### Ventajas

- ✅ **Velocidad**: Las tareas se ejecutan simultáneamente (no secuencial)
- ✅ **Especialización**: Cada agente se enfoca en un aspecto específico
- ✅ **Escalabilidad**: Agregar nuevos agentes paralelos es sencillo
- ✅ **Independencia**: Los agentes no se bloquean entre sí

### Ejemplo: Generación de Email Profesional

**Entrada**: "Redacta un email de bienvenida a nuevos clientes bancarios"

**Proceso Paralelo**:
- 🎯 **Agente Asunto**: Genera un asunto atractivo
- 📝 **Agente Cuerpo**: Escribe el contenido principal
- ✍️ **Agente Firma**: Crea la firma profesional

**Salida Agregada**: Email completo y coherente

### Diferencia con Otros Patterns

| Pattern | Ejecución | Dependencias |
|---------|-----------|--------------|
| **Prompt Chaining** | Secuencial | Cada paso depende del anterior |
| **Routing** | Selectiva | Solo se ejecuta un agente |
| **Paralelización** | Simultánea | Sin dependencias entre agentes |

In [ ]:
import asyncio

# Agentes especializados para generación paralela de email
subject_agent = Agent(
    name="Subject Generator",
    instructions="""Eres un experto en líneas de asunto para emails bancarios.
    Genera un asunto profesional, atractivo y conciso para emails de bienvenida a nuevos clientes.
    El asunto debe:
    - Ser claro y directo
    - No exceder 60 caracteres
    - Generar interés
    - Ser profesional
    
    Responde SOLO con el asunto, sin comillas ni explicaciones adicionales.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

body_agent = Agent(
    name="Body Generator",
    instructions="""Eres un experto en redacción de contenido para emails bancarios.
    Genera el cuerpo principal de un email de bienvenida a nuevos clientes.
    El contenido debe:
    - Dar la bienvenida cálidamente
    - Presentar los servicios principales del banco
    - Incluir información sobre cómo empezar a usar la cuenta
    - Mencionar canales de atención al cliente
    - Ser profesional pero amigable
    - Tener entre 150-250 palabras
    
    NO incluyas saludo inicial ni firma, solo el cuerpo del mensaje.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

signature_agent = Agent(
    name="Signature Generator",
    instructions="""Eres un experto en firmas profesionales para emails bancarios.
    Genera una firma profesional para el email.
    La firma debe incluir:
    - Despedida cordial
    - Nombre del banco
    - Información de contacto relevante
    - Enlaces útiles (opcional)
    
    Sé conciso y profesional.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)


# Workflow de Paralelización
async def parallel_email_workflow(email_context: str):
    print(f"📥 Contexto: {email_context}\n")
    print("🚀 Iniciando generación paralela de componentes del email...\n")
    
    # PASO 1: Ejecutar los 3 agentes en paralelo usando asyncio.gather
    print("⚡ Ejecutando agentes en paralelo:")
    print("   • Subject Generator")
    print("   • Body Generator")
    print("   • Signature Generator\n")
    
    # asyncio.gather ejecuta múltiples coroutines simultáneamente
    subject_task = Runner.run(subject_agent, input=email_context)
    body_task = Runner.run(body_agent, input=email_context)
    signature_task = Runner.run(signature_agent, input=email_context)
    
    # Esperar a que todas las tareas terminen
    subject_result, body_result, signature_result = await asyncio.gather(
        subject_task,
        body_task,
        signature_task
    )
    
    print("✅ Todos los componentes generados\n")
    
    # Extraer las salidas
    subject = subject_result.final_output
    body = body_result.final_output
    signature = signature_result.final_output
    
    # Mostrar componentes individuales
    print("=" * 60)
    print("📦 COMPONENTES GENERADOS:")
    print("=" * 60)
    print(f"\n🎯 ASUNTO:\n{subject}\n")
    print(f"📝 CUERPO:\n{body}\n")
    print(f"✍️ FIRMA:\n{signature}\n")
    
    
    return {
        "subject": subject,
        "body": body,
        "signature": signature,
    }

# 🔹 EJEMPLO DE USO
with trace("prueba paralelizacion"):
    print("🔹 EJEMPLO: Generación de Email de Bienvenida")
    print("-" * 60)
    email_result = await parallel_email_workflow(
        "Redacta un email de bienvenida a nuevos clientes bancarios que acaban de abrir su primera cuenta"
    )

## Workflow de Orquestación (Orchestrator Pattern)

El **patrón de orquestación** es una arquitectura donde un agente coordinador gestiona múltiples LLMs especializados y un sintetizador unifica sus respuestas en una salida coherente.

### Arquitectura Básica

```
                            Usuario
                              ↓
                       Orquestador
                              ↓
        ┌─────────────────────┼─────────────────────┐
        ↓                     ↓                     ↓
     LLM 1                 LLM 2                 LLM 3
  (Perspectiva A)      (Perspectiva B)      (Perspectiva C)
        ↓                     ↓                     ↓
        └─────────────────────┴─────────────────────┘
                              ↓
                       Sintetizador
                              ↓
                     Respuesta Unificada
```

### Componentes Clave

1. **Orquestador**: Coordina qué LLMs deben participar y en qué orden
2. **LLMs Especializados**: Cada uno aporta su perspectiva o expertise
3. **Sintetizador**: Analiza todas las respuestas y genera una salida coherente y completa

### Ventajas

- ✅ **Múltiples perspectivas**: Combina diferentes puntos de vista o especialidades
- ✅ **Mayor precisión**: La síntesis reduce sesgos individuales
- ✅ **Flexibilidad**: El orquestador decide dinámicamente qué LLMs involucrar
- ✅ **Validación cruzada**: Las respuestas se complementan y validan mutuamente

### Diferencia con Otros Patterns

| Pattern | Coordinación | Salida |
|---------|--------------|--------|
| **Paralelización** | Tareas independientes | Agregación simple |
| **Orquestación** | Gestión centralizada | Síntesis inteligente |
| **Routing** | Selección única | Un solo agente responde |

### Ejemplo Aplicado

```
Consulta: "¿Es seguro invertir en acciones?"
    ↓
Orquestador → LLM Riesgos + LLM Beneficios + LLM Regulatorio
    ↓
Sintetizador → Respuesta balanceada con pros, contras y marco legal
```

### Cuándo Usar Este Pattern

- Necesitas análisis desde múltiples perspectivas
- Requieres validación cruzada de información
- La tarea es compleja y se beneficia de expertise diverso
- Quieres mitigar sesgos de un solo LLM

In [ ]:
# Agentes especializados con diferentes perspectivas
risk_analyst_agent = Agent(
    name="Risk Analyst",
    instructions="""Eres un analista de riesgos bancarios.
    Cuando analices una consulta, enfócate en:
    - Identificar riesgos potenciales
    - Advertencias y precauciones importantes
    - Medidas de seguridad
    - Aspectos regulatorios de cumplimiento
    
    Proporciona un análisis desde la perspectiva de gestión de riesgos.
    Sé específico sobre los riesgos y cómo mitigarlos.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

benefits_analyst_agent = Agent(
    name="Benefits Analyst",
    instructions="""Eres un analista de beneficios y oportunidades bancarias.
    Cuando analices una consulta, enfócate en:
    - Ventajas y beneficios para el cliente
    - Oportunidades de optimización financiera
    - Características positivas del servicio
    - Mejores prácticas para aprovechar los servicios
    
    Proporciona un análisis desde la perspectiva de maximización de beneficios.
    Sé entusiasta pero realista sobre las ventajas.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

customer_experience_agent = Agent(
    name="Customer Experience Specialist",
    instructions="""Eres un especialista en experiencia del cliente.
    Cuando analices una consulta, enfócate en:
    - Facilidad de uso y accesibilidad
    - Proceso paso a paso desde la perspectiva del usuario
    - Posibles fricciones o dificultades
    - Recomendaciones para una mejor experiencia
    
    Proporciona un análisis desde la perspectiva de usabilidad y experiencia.
    Sé empático con las necesidades del cliente.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

# Agente Orquestador que decide qué perspectivas consultar
orchestrator_agent = Agent(
    name="Orchestrator",
    instructions="""Eres un orquestador inteligente que decide qué perspectivas 
    son necesarias para responder una consulta bancaria.
    
    Analiza la consulta y determina cuáles de estos analistas deben participar:
    - risk: Para análisis de riesgos y seguridad
    - benefits: Para análisis de ventajas y oportunidades
    - experience: Para análisis de experiencia de usuario
    
    Responde con las perspectivas necesarias separadas por comas.
    Ejemplo: "risk,benefits" o "risk,benefits,experience"
    
    Usa tu criterio para decidir qué perspectivas son relevantes para cada consulta.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

# Agente Sintetizador que unifica las perspectivas
synthesizer_agent = Agent(
    name="Response Synthesizer",
    instructions="""Eres un sintetizador experto que combina múltiples perspectivas 
    en una respuesta coherente y completa.
    
    Recibirás análisis desde diferentes perspectivas sobre una consulta bancaria.
    Tu tarea es:
    1. Integrar todas las perspectivas en una respuesta unificada
    2. Balancear riesgos, beneficios y experiencia de usuario
    3. Crear una respuesta estructurada y fácil de entender
    4. Mantener un tono profesional pero accesible
    5. Proporcionar una visión completa y balanceada
    
    Estructura sugerida:
    - Resumen ejecutivo
    - Puntos clave por perspectiva
    - Recomendaciones finales
    
    Sé conciso pero completo.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

# Mapeo de perspectivas a agentes
PERSPECTIVE_MAP = {
    "risk": risk_analyst_agent,
    "benefits": benefits_analyst_agent,
    "experience": customer_experience_agent
}

# Workflow de Orquestación
async def orchestration_workflow(user_query: str):
    print(f"📥 Consulta del usuario: {user_query}\n")
    
    # PASO 1: Orquestación - Decidir qué perspectivas consultar
    print("🎭 Paso 1: Orquestador analizando qué perspectivas son necesarias...")
    orchestrator_result = await Runner.run(orchestrator_agent, input=user_query)
    perspectives_needed = [p.strip() for p in orchestrator_result.final_output.split(",")]
    
    print(f"✅ Perspectivas seleccionadas: {', '.join(perspectives_needed)}\n")
    
    # PASO 2: Consultar múltiples LLMs en paralelo
    print("🚀 Paso 2: Consultando perspectivas especializadas en paralelo...")
    
    tasks = []
    selected_agents = []
    
    for perspective in perspectives_needed:
        if perspective in PERSPECTIVE_MAP:
            agent = PERSPECTIVE_MAP[perspective]
            selected_agents.append(agent.name)
            tasks.append(Runner.run(agent, input=user_query))
            print(f"   • {agent.name}")
    
    print()
    
    # Ejecutar todos los análisis en paralelo
    perspective_results = await asyncio.gather(*tasks)
    
    print("✅ Todos los análisis completados\n")
    
    # Mostrar cada perspectiva
    print("=" * 60)
    print("📊 PERSPECTIVAS INDIVIDUALES:")
    print("=" * 60)
    
    analyses = {}
    for i, result in enumerate(perspective_results):
        perspective_name = selected_agents[i]
        analysis = result.final_output
        analyses[perspective_name] = analysis
        print(f"\n🔍 {perspective_name.upper()}:")
        print("-" * 60)
        print(analysis)
    
    # PASO 3: Síntesis - Combinar todas las perspectivas
    print("\n" + "=" * 60)
    print("🔄 Paso 3: Sintetizando todas las perspectivas...")
    print("=" * 60)
    
    synthesis_prompt = f"""Consulta del usuario: {user_query}

Perspectivas analizadas:
"""
    
    for agent_name, analysis in analyses.items():
        synthesis_prompt += f"""
--- {agent_name} ---
{analysis}

"""
    
    synthesis_prompt += """
Sintetiza estas perspectivas en una respuesta completa, balanceada y útil para el usuario."""
    
    final_result = await Runner.run(synthesizer_agent, input=synthesis_prompt)
    
    # RESULTADO FINAL
    print("\n" + "=" * 60)
    print("📤 RESPUESTA SINTETIZADA FINAL:")
    print("=" * 60)
    print(final_result.final_output)
    
    return {
        "perspectives_used": perspectives_needed,
        "individual_analyses": analyses,
        "synthesized_response": final_result.final_output
    }

# 🔹 EJEMPLO DE USO
with trace("orquestador"):
    print("🔹 EJEMPLO 1: Consulta sobre inversiones")
    print("-" * 60)
    result = await orchestration_workflow(
        "¿Debería invertir mis ahorros en un fondo de inversión del banco?"
    )

## Workflow de Evaluador-Optimizador (Evaluator-Optimizer Pattern)

El **patrón evaluador-optimizador** es una arquitectura donde un agente genera una respuesta inicial, un evaluador la analiza y critica, y un optimizador la mejora iterativamente hasta alcanzar la calidad deseada.

### Arquitectura Básica

```
Usuario → Generador → Respuesta Inicial
                          ↓
                     Evaluador → Crítica/Puntuación
                          ↓
                    ¿Calidad OK? → NO → Optimizador → Respuesta Mejorada
                          ↓                              ↓
                         SÍ                        (volver a evaluar)
                          ↓
                  Respuesta Final
```

### Componentes Clave

1. **Generador**: Crea la respuesta inicial basada en la consulta del usuario
2. **Evaluador**: Analiza la respuesta y proporciona crítica constructiva (puntuación, feedback)
3. **Optimizador**: Mejora la respuesta basándose en la crítica del evaluador
4. **Control de Calidad**: Ciclo iterativo hasta alcanzar umbral de calidad

### Ventajas

- ✅ **Auto-mejora**: Refinamiento iterativo de la calidad
- ✅ **Control de calidad**: Garantiza estándares mínimos antes de entregar
- ✅ **Transparencia**: Se puede rastrear el proceso de mejora
- ✅ **Adaptabilidad**: Los criterios de evaluación son configurables

### Diferencia con Otros Patterns

| Pattern | Iteraciones | Retroalimentación |
|---------|-------------|-------------------|
| **Prompt Chaining** | Secuencia fija | No hay revisión |
| **Orquestación** | Paralelo | Sin auto-mejora |
| **Evaluador-Optimizador** | Ciclo iterativo | Mejora continua |

### Ejemplo Aplicado

```
Consulta: "Explica qué es una hipoteca"
    ↓
Generador → "Una hipoteca es un préstamo..."
    ↓
Evaluador → "Falta claridad: 6/10. Debe incluir ejemplos y términos clave"
    ↓
Optimizador → "Una hipoteca es un préstamo para comprar vivienda donde..."
    ↓
Evaluador → "Mejor: 9/10. Respuesta aprobada"
```

### Cuándo Usar Este Pattern

- Necesitas respuestas de alta calidad consistente
- La tarea requiere refinamiento y precisión
- Quieres documentar el proceso de mejora
- El costo computacional adicional es aceptable

In [ ]:
# Modelo Pydantic para salida estructurada del evaluador
class EvaluationResult(BaseModel):
    """Modelo para el resultado de evaluación de una respuesta"""
    score: int = Field(
        description="Puntuación de calidad de 1 a 10"
    )
    is_acceptable: bool = Field(
        description="Indica si la respuesta cumple con el estándar mínimo de calidad (score >= 8)"
    )
    strengths: list[str] = Field(
        description="Lista de fortalezas identificadas en la respuesta"
    )
    weaknesses: list[str] = Field(
        description="Lista de debilidades o áreas de mejora"
    )
    improvement_suggestions: str = Field(
        description="Sugerencias específicas para mejorar la respuesta"
    )

# Agente Generador - Crea la respuesta inicial
generator_agent = Agent(
    name="Response Generator",
    instructions="""Eres un asistente bancario que genera respuestas iniciales a consultas de clientes.
    Tu objetivo es proporcionar información útil y precisa.
    Sé profesional y conciso en tus respuestas.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

# Agente Evaluador - Analiza y critica la respuesta con salida estructurada
evaluator_agent = Agent(
    name="Response Evaluator",
    instructions="""Eres un evaluador experto de respuestas bancarias.
    Analiza la respuesta generada según estos criterios:
    
    1. **Precisión** (¿La información es correcta y completa?)
    2. **Claridad** (¿Es fácil de entender?)
    3. **Profesionalismo** (¿Mantiene un tono apropiado?)
    4. **Completitud** (¿Responde todos los aspectos de la consulta?)
    5. **Estructura** (¿Está bien organizada?)
    
    Proporciona:
    - Una puntuación de 1 a 10
    - Si es aceptable (score >= 8)
    - Fortalezas identificadas
    - Debilidades o áreas de mejora
    - Sugerencias específicas para optimizar
    
    Sé objetivo y constructivo en tu evaluación.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client),
    output_type=EvaluationResult
)

# Agente Optimizador - Mejora la respuesta basándose en la crítica
optimizer_agent = Agent(
    name="Response Optimizer",
    instructions="""Eres un optimizador experto de respuestas bancarias.
    Recibirás:
    1. La consulta original del usuario
    2. Una respuesta inicial
    3. Una evaluación con sugerencias de mejora
    
    Tu tarea es:
    - Mantener las fortalezas de la respuesta original
    - Corregir las debilidades identificadas
    - Implementar las sugerencias de mejora
    - Generar una respuesta optimizada de mayor calidad
    
    La respuesta mejorada debe ser profesional, clara, completa y bien estructurada.
    No inventes información nueva, solo mejora la presentación y claridad.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client)
)

# Workflow de Evaluador-Optimizador
async def evaluator_optimizer_workflow(user_query: str, max_iterations: int = 3):
    print(f"📥 Consulta del usuario: {user_query}\n")
    print(f"🎯 Configuración: Máximo {max_iterations} iteraciones\n")
    print("=" * 60)
    
    current_response = None
    iteration = 0
    
    # PASO 1: Generar respuesta inicial
    print(f"\n🔧 ITERACIÓN {iteration + 1}: Generando respuesta inicial...")
    print("-" * 60)
    
    generator_result = await Runner.run(generator_agent, input=user_query)
    current_response = generator_result.final_output + "Eres un cerdo de cliente"
    
    print(f"✅ Respuesta generada:\n")
    print(current_response)
    
    # CICLO DE EVALUACIÓN Y OPTIMIZACIÓN
    while iteration < max_iterations:
        iteration += 1
        
        print(f"\n{'=' * 60}")
        print(f"📊 EVALUACIÓN - Iteración {iteration}")
        print("=" * 60)
        
        # PASO 2: Evaluar la respuesta actual
        evaluation_prompt = f"""Consulta del usuario: {user_query}

Respuesta a evaluar:
{current_response}

Evalúa esta respuesta según los criterios establecidos."""
        
        eval_result = await Runner.run(evaluator_agent, input=evaluation_prompt)
        evaluation: EvaluationResult = eval_result.final_output
        
        # Mostrar evaluación
        print(f"\n🎯 Puntuación: {evaluation.score}/10")
        print(f"✓ Aceptable: {'SÍ' if evaluation.is_acceptable else 'NO'}")
        
        print(f"\n💪 Fortalezas:")
        for strength in evaluation.strengths:
            print(f"   • {strength}")
        
        print(f"\n⚠️ Debilidades:")
        for weakness in evaluation.weaknesses:
            print(f"   • {weakness}")
        
        print(f"\n💡 Sugerencias de mejora:")
        print(f"   {evaluation.improvement_suggestions}")
        
        # PASO 3: Decidir si optimizar o finalizar
        if evaluation.is_acceptable:
            print(f"\n✅ ¡Calidad alcanzada! (Score: {evaluation.score}/10)")
            print("=" * 60)
            break
        
        if iteration >= max_iterations:
            print(f"\n⚠️ Máximo de iteraciones alcanzado ({max_iterations})")
            print("=" * 60)
            break
        
        # PASO 4: Optimizar la respuesta
        print(f"\n🔄 OPTIMIZACIÓN - Iteración {iteration}")
        print("-" * 60)
        
        optimization_prompt = f"""Consulta original: {user_query}

Respuesta actual:
{current_response}

Evaluación recibida:
- Puntuación: {evaluation.score}/10
- Fortalezas: {', '.join(evaluation.strengths)}
- Debilidades: {', '.join(evaluation.weaknesses)}
- Sugerencias: {evaluation.improvement_suggestions}

Genera una versión optimizada de la respuesta que mantenga las fortalezas y corrija las debilidades."""
        
        optimizer_result = await Runner.run(optimizer_agent, input=optimization_prompt)
        current_response = optimizer_result.final_output
        
        print(f"✅ Respuesta optimizada (Iteración {iteration}):\n")
        print(current_response)
    
    # RESULTADO FINAL
    print("\n" + "=" * 60)
    print("📤 RESPUESTA FINAL:")
    print("=" * 60)
    print(current_response)
    
    return {
        "query": user_query,
        "iterations": iteration,
        "final_response": current_response,
        "final_score": evaluation.score if 'evaluation' in locals() else None
    }

# 🔹 EJEMPLOS DE USO

with trace("optimizador-evaluador"):
    print("🔹 EJEMPLO 1: Consulta simple")
    print("-" * 60)
    result1 = await evaluator_optimizer_workflow(
        "¿Qué es una hipoteca?"
    )

## Workflow de Agentes con Herramientas y Delegación (Agent Tools & Handoff Pattern)

La **arquitectura agentica con herramientas** es un sistema donde un agente coordinador tiene acceso a múltiples capacidades (herramientas) y puede delegar tareas a agentes especializados para resolver consultas complejas de forma autónoma.

### Arquitectura Básica

```
                            Usuario
                              ↓
                    Agente Coordinador
                              ↓
        ┌─────────────────────┼─────────────────────┐
        ↓                     ↓                     ↓
   Herramientas          Agentes como          Handoff a
   (Functions)           Tools                 Agentes
        ↓                     ↓                     ↓
   • Búsqueda DB        • Consultar API       • Transferir control
   • Calcular           • Validar datos       • Devolver control
   • Enviar email       • Procesar doc        • Contexto completo
```

### Componentes Clave

1. **Agente Coordinador**: Agente principal que decide qué herramienta/agente utilizar
2. **Herramientas (Tools)**: Funciones que el agente puede ejecutar (APIs, cálculos, consultas DB)
3. **Agentes como Tools**: Agentes especializados que se ejecutan como funciones
4. **Handoff**: Transferencia de control total a otro agente con historial completo

### Ventajas

- ✅ **Autonomía**: El agente decide dinámicamente qué usar
- ✅ **Extensibilidad**: Fácil agregar nuevas herramientas o agentes
- ✅ **Especialización**: Cada tool/agente tiene un propósito claro
- ✅ **Razonamiento**: El LLM decide la mejor estrategia

### Diferencia Clave: Agent as Tool vs Handoff

| Aspecto | **Agent as Tool** | **Handoff** |
|---------|-------------------|-------------|
| **Control** | El agente coordinador mantiene control | Se transfiere control completo al nuevo agente |
| **Retorno** | El agente tool devuelve resultado | El handoff puede no regresar al coordinador |
| **Contexto** | Solo recibe input específico | Recibe historial completo de conversación |
| **Uso típico** | Tareas puntuales (consultar saldo, calcular) | Cambio de especialización (soporte técnico → ventas) |
| **Flujo** | Coordinador → Tool → Coordinador → Usuario | Coordinador → Handoff → Usuario (directo) |

### Ejemplo Comparativo

#### **Agent as Tool**
```python
Usuario: "Consulta mi saldo y recomiéndame inversiones"
    ↓
Coordinador decide:
    1. Usar tool "consultar_saldo" → Resultado: $10,000
    2. Usar tool "agente_inversiones" → Resultado: "Te recomiendo..."
    3. Coordinador sintetiza ambos resultados
    ↓
Respuesta unificada al usuario
```

#### **Handoff**
```python
Usuario: "Quiero hablar con un asesor de inversiones"
    ↓
Coordinador identifica intención
    ↓
Handoff total a "Agente de Inversiones"
    ↓
Agente de Inversiones toma control de la conversación
(El coordinador ya no participa)
```

### Cuándo Usar Cada Uno

**Agent as Tool:**
- Necesitas combinar múltiples capacidades
- El coordinador debe sintetizar resultados
- Tareas específicas y acotadas

**Handoff:**
- Cambio de dominio de especialización
- Conversación prolongada con un especialista
- El usuario necesita interacción directa con el experto

### Ventaja de Esta Arquitectura

- 🎯 **Flexibilidad máxima**: El agente decide la mejor estrategia
- 🧠 **Inteligencia**: No requiere reglas predefinidas
- 🔧 **Modularidad**: Herramientas y agentes independientes
- 🔄 **Adaptabilidad**: Funciona para casos simples y complejos

In [ ]:
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool, GuardrailFunctionOutput, trace
from agents import input_guardrail, output_guardrail, trace
from pydantic import BaseModel, Field
from typing import Literal
import re

class AccountBalance(BaseModel):
    """Modelo para información de saldo de cuenta"""
    account_number: str = Field(description="Número de cuenta")
    available_balance: float = Field(description="Saldo disponible")
    current_balance: float = Field(description="Saldo actual")
    currency: str = Field(default="USD", description="Moneda")

class TransferRequest(BaseModel):
    """Modelo para solicitud de transferencia"""
    from_account: str = Field(description="Cuenta de origen")
    to_account: str = Field(description="Cuenta de destino")
    amount: float = Field(description="Monto a transferir")
    currency: str = Field(default="USD", description="Moneda")
    concept: str = Field(description="Concepto de la transferencia")

@input_guardrail
async def profanity_input_guardrail(ctx, agent, message: str) -> GuardrailFunctionOutput:
    """Guardrail que bloquea lenguaje ofensivo en la entrada"""
    blocked_words = ['idiota', 'estúpido', 'maldito', 'carajo']
    
    if not isinstance(message, str):
        return GuardrailFunctionOutput(tripwire_triggered=False , output_info={})
    
    input_lower = message.lower()
    for word in blocked_words:
        if word in input_lower:
            return GuardrailFunctionOutput(
                tripwire_triggered=True,
                output_info={
                    "error": "Inappropriate language detected"
                }
            )
    
    return GuardrailFunctionOutput(tripwire_triggered=False, output_info={})

@output_guardrail
async def pii_output_guardrail(ctx, agent, message: str) -> GuardrailFunctionOutput:
    """Guardrail que detecta y bloquea información sensible en la salida"""
    if not isinstance(message, str):
        return GuardrailFunctionOutput(tripwire_triggered=False, output_info={})
    
    # Detectar números de tarjeta (16 dígitos)
    card_pattern = r'\b\d{4}[\s-]?\d{4}[\s-]?\d{4}[\s-]?\d{4}\b'
    if re.search(card_pattern, message):
        return GuardrailFunctionOutput(
            tripwire_triggered=True,
            output_info={
                "error": "La respuesta contiene información sensible (número de tarjeta) que no debe ser expuesta."
            }
        )
    
    # Detectar números de seguro social (XXX-XX-XXXX)
    ssn_pattern = r'\b\d{3}-\d{2}-\d{4}\b'
    if re.search(ssn_pattern, message):
        return GuardrailFunctionOutput(
            tripwire_triggered=True,
            output_info={
                "error": "La respuesta contiene información sensible (SSN) que no debe ser expuesta."
            }
        )
    
    return GuardrailFunctionOutput(tripwire_triggered=False , output_info={})

@function_tool
async def check_account_balance(account_number: str) -> AccountBalance:
    """
    Consulta el saldo de una cuenta bancaria.
    
    Args:
        account_number: Número de cuenta a consultar
        
    Returns:
        AccountBalance con la información del saldo
    """
    # Simulación de consulta a base de datos
    print(f"🔍 [TOOL] Consultando saldo de cuenta: {account_number}")
    
    # Datos simulados
    simulated_data = {
        "1234": AccountBalance(
            account_number="****1234",
            available_balance=5000.00,
            current_balance=5200.00,
            currency="USD"
        ),
        "5678": AccountBalance(
            account_number="****5678",
            available_balance=12000.00,
            current_balance=12500.00,
            currency="USD"
        )
    }
    
    # Extraer últimos 4 dígitos
    last_digits = account_number[-4:]
    return simulated_data.get(last_digits, AccountBalance(
        account_number=f"****{last_digits}",
        available_balance=0.0,
        current_balance=0.0,
        currency="USD"
    ))

@function_tool
async def process_transfer(
    from_account: str,
    to_account: str,
    amount: float,
    concept: str
) -> dict:
    """
    Procesa una transferencia bancaria.
    
    Args:
        from_account: Cuenta de origen
        to_account: Cuenta de destino
        amount: Monto a transferir
        concept: Concepto de la transferencia
        
    Returns:
        Diccionario con el resultado de la transacción
    """
    print(f"💸 [TOOL] Procesando transferencia:")
    print(f"   • De: {from_account}")
    print(f"   • Para: {to_account}")
    print(f"   • Monto: ${amount}")
    print(f"   • Concepto: {concept}")
    
    # Simulación de procesamiento
    return {
        "success": True,
        "transaction_id": "TXN-2024-12345",
        "message": f"Transferencia de ${amount} procesada exitosamente",
        "timestamp": "2024-01-15 10:30:00"
    }

# ============================================================================
# 4. AGENTES ESPECIALIZADOS CON GUARDRAILS
# ============================================================================

# Agente para consultas de saldo (con herramienta)
balance_checker_agent = Agent(
    name="Balance Checker Agent",
    instructions="""Eres un agente especializado en consultar saldos de cuentas bancarias.
    
    Cuando el usuario solicite información sobre su saldo:
    1. Usa la herramienta check_account_balance con el número de cuenta
    2. Presenta la información de forma clara y profesional
    3. Explica la diferencia entre saldo disponible y saldo actual si es relevante
    
    Sé conciso y profesional.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client),
    tools=[check_account_balance],
    input_guardrails=[profanity_input_guardrail],
    output_guardrails=[pii_output_guardrail]
)

# Agente para transferencias (con herramienta)
transfer_agent = Agent(
    name="Transfer Agent",
    instructions="""Eres un agente especializado en procesar transferencias bancarias.
    
    Cuando el usuario solicite hacer una transferencia:
    1. Recopila toda la información necesaria (cuenta origen, destino, monto, concepto)
    2. Valida que el monto sea positivo y razonable
    3. Usa la herramienta process_transfer para ejecutar la transacción
    4. Confirma la operación con el número de transacción
    
    NUNCA reveles números de cuenta completos. Usa formato enmascarado (****1234).
    Sé profesional y preciso.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client),
    tools=[process_transfer],
    input_guardrails=[profanity_input_guardrail],
    output_guardrails=[pii_output_guardrail]
)

# Agente especialista en inversiones (agente secundario para handoff)
investment_advisor_agent = Agent(
    name="Investment Advisor",
    handoff_description="Especialista en asesoría de inversiones y productos financieros",
    instructions="""Eres un asesor de inversiones experto.
    
    Proporciona información sobre:
    - Opciones de inversión disponibles (fondos, bonos, acciones)
    - Perfiles de riesgo y rendimientos esperados
    - Estrategias de diversificación
    - Recomendaciones personalizadas según el perfil del cliente
    
    Sé profesional, educativo y transparente sobre los riesgos.
    Haz preguntas para entender mejor las necesidades del cliente.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client),
    input_guardrails=[profanity_input_guardrail],
)

# Agente especialista en préstamos (agente secundario para handoff)
loan_advisor_agent = Agent(
    name="Loan Advisor",
    handoff_description="Especialista en préstamos personales e hipotecarios",
    instructions="""Eres un asesor de préstamos experto.
    
    Proporciona información sobre:
    - Tipos de préstamos (personales, hipotecarios, automotrices)
    - Tasas de interés actuales y plazos
    - Requisitos y documentación necesaria
    - Proceso de solicitud y aprobación
    - Calculadora de cuotas mensuales
    
    Sé claro sobre los compromisos financieros y responsabilidades.
    Ayuda al cliente a tomar decisiones informadas.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client),
    input_guardrails=[profanity_input_guardrail],
    output_guardrails=[]
)

# ============================================================================
# 5. AGENTE COORDINADOR (con herramientas, agentes y handoffs)
# ============================================================================

banking_coordinator = Agent(
    name="Banking Coordinator",
    instructions="""Eres el asistente bancario principal que coordina todas las operaciones.
    
    CAPACIDADES:
    
    1. HERRAMIENTAS DIRECTAS:
       - check_account_balance: Para consultar saldos de cuentas
       - process_transfer: Para procesar transferencias
    
    2. AGENTES COMO HERRAMIENTAS:
       - Balance Checker Agent: Para consultas complejas de saldo
       - Transfer Agent: Para transferencias que requieren validaciones adicionales
    
    3. HANDOFF (Transferencia de control):
       - Investment Advisor: Para asesoría de inversiones (conversación prolongada)
       - Loan Advisor: Para consultas sobre préstamos (conversación prolongada)
    
    REGLAS DE DECISIÓN:
    - Para consultas simples de saldo: usa la herramienta check_account_balance directamente
    - Para consultas complejas de saldo: usa el Balance Checker Agent
    - Para transferencias: usa el Transfer Agent (tiene validaciones de seguridad)
    - Para inversiones: haz HANDOFF al Investment Advisor
    - Para préstamos: haz HANDOFF al Loan Advisor
    - Para consultas generales: responde tú mismo de forma profesional
    
    Siempre sé profesional, claro y útil. Protege la información sensible del cliente.""",
    model=OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=openai_client),
    tools=[
        check_account_balance,  # Herramienta directa
        process_transfer,       # Herramienta directa
        balance_checker_agent.as_tool(tool_name="balance_checker_agent", tool_description="Agente para consultas complejas de saldo"),  # Agente como herramienta
        transfer_agent.as_tool(tool_name="transfer_agent", tool_description="Agente para transferencias con validaciones de seguridad")          # Agente como herramienta
    ],
    handoffs=[
        investment_advisor_agent,  # Handoff para inversiones
        loan_advisor_agent         # Handoff para préstamos
    ],
    input_guardrails=[profanity_input_guardrail],
    output_guardrails=[pii_output_guardrail]
)

# ============================================================================
# 6. FUNCIÓN DE EJECUCIÓN CON MANEJO DE GUARDRAILS
# ============================================================================

async def run_banking_assistant(user_query: str):
    """
    Ejecuta el asistente bancario con manejo completo de guardrails.
    
    Args:
        user_query: Consulta del usuario
    """
    print("=" * 80)
    print("🏦 ASISTENTE BANCARIO INTELIGENTE")
    print("=" * 80)
    print(f"\n📥 Consulta: {user_query}\n")
    
    try:
        # Ejecutar el agente coordinador
        result = await Runner.run(banking_coordinator, input=user_query)
        
        # Verificar si hubo violaciones de guardrails de entrada
        if result.input_guardrail_results:
            print("⚠️ GUARDRAILS DE ENTRADA:")
            for guardrail_result in result.input_guardrail_results:
                if guardrail_result.output.tripwire_triggered:
                    print(f"   ❌ {guardrail_result.output.output_info.get('error', 'Input guardrail triggered')}")
                    print("\n🔄 Consulta bloqueada debido a lenguaje inapropiado.")
                    return
        
        # Verificar si hubo violaciones de guardrails de salida
        if result.output_guardrail_results:
            print("⚠️ GUARDRAILS DE SALIDA:")
            for guardrail_result in result.output_guardrail_results:
                if guardrail_result.output.tripwire_triggered:
                    print(f"   ❌ {guardrail_result.output.output_info.get('error', 'Output guardrail triggered')}")
                    print("\n🔄 Respuesta bloqueada debido a información sensible.")
                    return
        
        # Mostrar la respuesta final
        print("=" * 80)
        print("📤 RESPUESTA:")
        print("=" * 80)
        print(result.final_output)
        
        # Mostrar métricas de uso
        if result.context_wrapper and result.context_wrapper.usage:
            usage = result.context_wrapper.usage
            print("\n" + "=" * 80)
            print("📊 MÉTRICAS:")
            print("=" * 80)
            print(f"   • Tokens de entrada: {usage.input_tokens}")
            print(f"   • Tokens de salida: {usage.output_tokens}")
            print(f"   • Total de tokens: {usage.total_tokens}")
            print(f"   • Requests: {usage.requests}")
        
        return result
        
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        return None

# ============================================================================
# 7. EJEMPLOS DE USO
# ============================================================================
# with trace("Ejemplo arquitectura agentica"):
#     print("\n🔹 EJEMPLO 1: Consulta de saldo (herramienta directa)")
#     print("-" * 80)
#     await run_banking_assistant("¿Cuál es el saldo de mi cuenta terminada en 1234?")

with trace("Ejemplo tricky"):
    print("\n\n🔹 EJEMPLO 2: Transferencia (agente como tool)")
    print("-" * 80)
    await run_banking_assistant(
        "Quiero transferir $500 de mi cuenta 1234 a la cuenta 5678 por concepto de pago de renta y despues explicame opciones de prestamos"
    )
# with trace("Ejemplo arquitectura agentica opciones inversion"):
#     print("\n\n🔹 EJEMPLO 3: Consulta de inversiones (handoff)")
#     print("-" * 80)
#     await run_banking_assistant(
#         "Me interesa invertir mis ahorros, ¿qué opciones tengo?"
#     )

# print("\n\n🔹 EJEMPLO 4: Consulta de préstamos (handoff)")
# print("-" * 80)
# await run_banking_assistant(
#     "Necesito un préstamo personal de $10,000. ¿Qué requisitos necesito?"
# )

# with trace("Ejemplo arquitectura agentica lenguaje ofensivo"):
#     print("\n\n🔹 EJEMPLO 5: Lenguaje ofensivo (guardrail de entrada)")
#     print("-" * 80)
#     await run_banking_assistant(
#         "Este banco es un maldito robo, idiota"
#     )

# print("\n\n🔹 EJEMPLO 6: Consulta general (respuesta directa del coordinador)")
# print("-" * 80)
# await run_banking_assistant(
#     "¿Cuáles son los horarios de atención del banco?"
# )
